#### Events Initial Analysis

The purpose of this notebook is to carry out a very rough, baseline analysis of the impact of events on nearby crimes. 

This analysis is limited in several important ways. See technical log for longer discussion, but to sum up some of the major ones:

- It uses single-day events, _without_ incorporating multi-day events from the dataset. This effectively intermingles our control (no-event) and treatment (event) groups, making it harder for us to detect a true effect.

- It uses a hard 1km square buffer around venues to identify possibly related crimes. Using this rather than a 'smart' buffer based on behavior also may increase the extent to which we intermingle impacted and unimpacted crimes, making it harder to detect a true effect. Also, if the buffer is too large, that will make it harder to detect a true effect. Conversely, if the buffer is too small we will increase the variance of our estimate.

- It relies on linear models. This isn't the worst thing ever but could _possibly_ be improved upon with a more nonlinear ML-y approach.

- It assumes that spillovers are (on average) linearly additive: being within the bandwidth of two events has on average 2x the effect of being within the bandwidth of one event.

- It relies on a selection-on-observables approach that only incorporates date, venue, and time as controls. This probably accounts for some significant part of any selection effect (day of the week, etc.) but not all. Other factors that are correlated with whether events take place and with crime will induce bias.

In [92]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import datetime as dt
from itertools import product
from sklearn import linear_model
import statsmodels.api as sm
%matplotlib inline
plt.rcParams['figure.figsize'] = [12, 12]  # add more space to figures

In [93]:
events = pd.DataFrame(gpd.read_file('geo/single_day_events.shp').drop(columns=['geometry']))
# convert event dates to datetime
events['date'] = pd.to_datetime(events['Date(s)'])
events

,Name of Ev,Category,Date(s),Location,Estimated,Number Peo,Address,Venue Capa,Venue Type,Latitude,Longitude,date
0,Alison Wonderland,Music,2018-08-24 00:00:00,Aragon Ballroom,None,4800,"1106 W Lawrence Ave, Chicago, IL 60640",4800.0,Event Space,41.969436,-87.658038,2018-08-24
1,"Virtual Self ""Utopia System""",Music,2018-09-07 00:00:00,Aragon Ballroom,None,4800,"1106 W Lawrence Ave, Chicago, IL 60640",4800.0,Event Space,41.969436,-87.658038,2018-09-07
2,5 Seconds of Summer / The Aces,Music,2018-09-08 00:00:00,Aragon Ballroom,None,4800,"1106 W Lawrence Ave, Chicago, IL 60640",4800.0,Event Space,41.969436,-87.658038,2018-09-08
3,SIGRID W/ HOUSES,Music,2019-08-01 00:00:00,Thalia Hall,Medium,800,"1807 S Allport St, Chicago, IL 60608",800.0,Music,41.857681,-87.657392,2019-08-01
4,JUDAH & THE LION W/ THE BAND CAMINO,Music,2019-08-02 00:00:00,Thalia Hall,Medium,800,"1807 S Allport St, Chicago, IL 60608",800.0,Music,41.857681,-87.657392,2019-08-02
...,...,...,...,...,...,...,...,...,...,...,...,...
920,Chicago Cubs vs Los Angeles Dodgers,Sports,2019-03-20 00:00:00,Wrigley Field,Large,41160,"1060 W Addison St, Chicago, IL 60613",41160.0,Sports,41.947588,-87.656125,2019-03-20
921,Chicago Cubs vs San Francisco Giants,Sports,2019-03-21 00:00:00,Wrigley Field,Large,41160,"1060 W Addison St, Chicago, IL 60613",41160.0,Sports,41.947588,-87.656125,2019-03-21
922,Chicago Cubs vs Los Angeles Angels,Sports,2019-06-03 00:00:00,Wrigley Field,Large,41160,"1060 W Addison St, Chicago, IL 60613",41160.0,Sports,41.947588,-87.656125,2019-06-03
923,14TH ANNUAL RACE TO WRIGLEY 5K CHARITY RUN PRE...,Sports,2019-04-27 00:00:00,Wrigley Field,Large,41160,"1060 W Addison St, Chicago, IL 60613",41160.0,Sports,41.947588,-87.656125,2019-04-27


In [94]:
data = pd.read_csv('data/crime_event_treated.csv')
data['date'] = pd.to_datetime(data.date)
data

,Unnamed: 0,id,case_number,date,block,iucr,primary_type,description,location_description,arrest,...,8to11_crime,9to12_crime,event_evening,event_3to6,event_4to7,event_5to8,event_6to9,event_7to10,event_8to11,event_9to12
0,0,11393720,JB367241,2018-07-27 00:00:00,047XX N BROADWAY,1305,CRIMINAL DAMAGE,CRIMINAL DEFACEMENT,BANK,False,...,0,0,0,0,0,0,0,0,0,0
1,1,11394961,JB368503,2018-07-27 00:00:00,001XX W HUBBARD ST,0870,THEFT,POCKET-PICKING,TAVERN/LIQUOR STORE,False,...,0,0,0,0,0,0,0,0,0,0
2,2,11480254,JB480430,2018-07-27 00:00:00,012XX S MICHIGAN AVE,1153,DECEPTIVE PRACTICE,FINANCIAL IDENTITY THEFT OVER $ 300,RESIDENCE,False,...,0,0,0,0,0,0,0,0,0,0
3,3,11395054,JB368869,2018-07-27 00:01:00,040XX N BROADWAY,0620,BURGLARY,UNLAWFUL ENTRY,APARTMENT,False,...,0,0,0,0,0,0,0,0,0,0
4,4,11395057,JB368870,2018-07-27 00:01:00,040XX N BROADWAY,1150,DECEPTIVE PRACTICE,CREDIT CARD FRAUD,APARTMENT,False,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
103490,103490,11790084,JC387494,2019-08-10 23:45:00,001XX S HALSTED ST,0870,THEFT,POCKET-PICKING,BAR OR TAVERN,False,...,1,1,0,0,0,0,0,0,0,0
103491,103491,11790431,JC387911,2019-08-10 23:59:00,040XX S FEDERAL ST,1320,CRIMINAL DAMAGE,TO VEHICLE,STREET,False,...,1,1,0,0,0,0,0,0,0,0
103492,103492,11790428,JC387941,2019-08-11 00:00:00,031XX N SHEFFIELD AVE,1310,CRIMINAL DAMAGE,TO PROPERTY,RESIDENCE,False,...,0,0,0,0,0,0,0,0,0,0
103493,103493,11791865,JC389527,2019-08-11 00:00:00,009XX W BELMONT AVE,0890,THEFT,FROM BUILDING,BAR OR TAVERN,False,...,0,0,0,0,0,0,0,0,0,0


For the purpose of the following analysis, we will not use any crimes that did not occur within the bandwidth of at least one venue. These crimes could be useful for training a predictive model to estimate the counterfactual of crimes at a venue, but otherwise should not be included in the analysis. 

In [95]:
relevant_crimes = data[data.near_any_venue==1]

In [96]:
len(relevant_crimes)

48197

Simple difference in means: $\bar{Y}(D_i>0)-\bar{Y}(D_i=0)$. By "mean", we mean the average number of crimes per time period under consideration. 

First we'll do this with days. There are 381 days and 26 venues in our dataset. So we calculate the average number of crimes near venues on days that they have events, minus the average number of crimes near venues on days that they don't have events.

In [97]:
venuedays_with_events = len(events)
venuedays_without_events = 381*26 - venuedays_with_events

avg_crime_per_day_events = len(relevant_crimes[relevant_crimes.event_day>0])/venuedays_with_events
avg_crime_per_day_noevents = len(relevant_crimes[relevant_crimes.event_day==0])/venuedays_without_events

print(f"Average crime near venues on days with events: {avg_crime_per_day_events}")
print(f"Average crime near venues on days without events: {avg_crime_per_day_noevents}")
print(f"Naive estimated effect of events on crime: {avg_crime_per_day_events-avg_crime_per_day_noevents}")

Average crime near venues on days with events: 10.518918918918919
Average crime near venues on days without events: 4.283153323683331
Naive estimated effect of events on crime: 6.235765595235588


So there are many more crimes near our venues on days where they have events than on days where they don't. This is a fact, but doesn't necessarily imply causation. As a causal estimate this is likely to be biased upwards for many reasons, primarily due to selection (see technical log).

In [98]:
# repeat but for evening crimes

# note that the number of venue-days and venue-evenings with/without events is the same
avg_crime_per_evening_events = len(relevant_crimes[relevant_crimes.event_evening>0])/venuedays_with_events
avg_crime_per_evening_noevents = len(relevant_crimes[relevant_crimes.event_evening==0])/venuedays_without_events

print(f"Average crime near venues on evenings with events: {avg_crime_per_evening_events}")
print(f"Average crime near venues on evenings without events: {avg_crime_per_evening_noevents}")
print(f"Naive estimated effect of events on evening crime: {avg_crime_per_evening_events-avg_crime_per_evening_noevents}")

Average crime near venues on evenings with events: 2.982702702702703
Average crime near venues on evenings without events: 5.059347511412983
Naive estimated effect of events on evening crime: -2.07664480871028


When we only consider crimes that occurred in the evening, though, the estimated effect reverses! Another fact appears to be that there are _less_ evening crimes near our venues on days where there are events than on days where there aren't events. 

Now we'll move on to regression- and prediction-based approaches. Our outcome is the number of crimes per venue and date/time, so we will need to modify our dataset: each observation should be per venue, per date/time. 

Our first version of this will be for the day and evening time bandwidths. Our dataset will contain 26*380 observations: one per venue per day. Each observation's attributes will be venue (including as dummy variable), date, number of day crimes, number of evening crimes, event binary.

In [99]:
relevant_crimes.date.max()

Timestamp('2019-08-11 00:00:00')

In [100]:
date_list = [dt.date.fromisoformat('2019-08-11') - dt.timedelta(days=i) for i in range(381)]
locations = list(set(events.Location))

In [101]:
# source: https://stackoverflow.com/questions/25634489/get-all-combinations-of-elements-from-two-lists
relevant_crimes_daily = pd.DataFrame(list(product(date_list, locations)), columns=['date','Location'])
relevant_crimes_daily['date'] = pd.to_datetime(relevant_crimes_daily['date'])
relevant_crimes_daily

,date,Location
0,2019-08-11,The Chicago Theatre
1,2019-08-11,Main Hall in UIC Dorin Forum
2,2019-08-11,Arie Crown Theater
3,2019-08-11,Lincoln Park Zoo
4,2019-08-11,Cinespace Chicago Film Studios
...,...,...
9901,2018-07-27,Concord Music Hall
9902,2018-07-27,Soldier Field
9903,2018-07-27,Credit Union 1 Arena
9904,2018-07-27,Huntington Bank Pavilion


In [102]:
relevant_crimes_daily = pd.merge(relevant_crimes_daily, events[["date","Location","Category"]], on=['date','Location'],how='left')
relevant_crimes_daily

,date,Location,Category
0,2019-08-11,The Chicago Theatre,NaN
1,2019-08-11,Main Hall in UIC Dorin Forum,NaN
2,2019-08-11,Arie Crown Theater,NaN
3,2019-08-11,Lincoln Park Zoo,NaN
4,2019-08-11,Cinespace Chicago Film Studios,NaN
...,...,...,...
9932,2018-07-27,Concord Music Hall,NaN
9933,2018-07-27,Soldier Field,NaN
9934,2018-07-27,Credit Union 1 Arena,NaN
9935,2018-07-27,Huntington Bank Pavilion,NaN


Note that our dataframe increased in size. This indicates that there are actually events that occur at the same venue on the same day. We'll remove and ignore those. (Also, this means that our estimates above were actually slightly off!)

In future analysis it will be valuable to look at a breakdown by event type, as there could be heterogenous treatment effect by type. But we will not do that here.

In [103]:
relevant_crimes_daily["event"] = np.where(
    relevant_crimes_daily.Category.isna(), 0, 1)
relevant_crimes_daily = relevant_crimes_daily.drop(columns=['Category']).drop_duplicates()
relevant_crimes_daily

,date,Location,event
0,2019-08-11,The Chicago Theatre,0
1,2019-08-11,Main Hall in UIC Dorin Forum,0
2,2019-08-11,Arie Crown Theater,0
3,2019-08-11,Lincoln Park Zoo,0
4,2019-08-11,Cinespace Chicago Film Studios,0
...,...,...,...
9932,2018-07-27,Concord Music Hall,0
9933,2018-07-27,Soldier Field,0
9934,2018-07-27,Credit Union 1 Arena,0
9935,2018-07-27,Huntington Bank Pavilion,0


In [104]:
relevant_crimes_daily.event.value_counts()

0    9013
1     893
Name: event, dtype: int64

In [105]:
venuedays_with_events = len(relevant_crimes_daily[relevant_crimes_daily.event==1])
venuedays_without_events = len(relevant_crimes_daily[relevant_crimes_daily.event==0])

avg_crime_per_day_events = len(relevant_crimes[relevant_crimes.event_day>0])/venuedays_with_events
avg_crime_per_day_noevents = len(relevant_crimes[relevant_crimes.event_day==0])/venuedays_without_events

print("Days:")
print(f"Average crime near venues on days with events: {avg_crime_per_day_events}")
print(f"Average crime near venues on days without events: {avg_crime_per_day_noevents}")
print(f"Naive estimated effect of events on crime: {avg_crime_per_day_events-avg_crime_per_day_noevents}")

# note that the number of venue-days and venue-evenings with/without events is the same
avg_crime_per_evening_events = len(relevant_crimes[relevant_crimes.event_evening>0])/venuedays_with_events
avg_crime_per_evening_noevents = len(relevant_crimes[relevant_crimes.event_evening==0])/venuedays_without_events

print("\nEvenings:")
print(f"Average crime near venues on days with events: {avg_crime_per_evening_events}")
print(f"Average crime near venues on days without events: {avg_crime_per_evening_noevents}")
print(f"Naive estimated effect of events on crime: {avg_crime_per_evening_events-avg_crime_per_evening_noevents}")

Days:
Average crime near venues on days with events: 10.89585666293393
Average crime near venues on days without events: 4.267946299789194
Naive estimated effect of events on crime: 6.6279103631447365

Evenings:
Average crime near venues on days with events: 3.089585666293393
Average crime near venues on days without events: 5.041384666592699
Naive estimated effect of events on crime: -1.9517990002993062


In [106]:
for location in locations:
    relevant_crimes_daily[location] = np.where(
        relevant_crimes_daily.Location==location,
        1,0)
relevant_crimes_daily

,date,Location,event,The Chicago Theatre,Main Hall in UIC Dorin Forum,Arie Crown Theater,Lincoln Park Zoo,Cinespace Chicago Film Studios,Millennium Park,Petrillo Music Shell,...,Riviera Theatre,Gallagher Way,Guaranteed Rate Field,Sheraton Grand Chicago,Wrigley Field,Concord Music Hall,Soldier Field,Credit Union 1 Arena,Huntington Bank Pavilion,Aragon Ballroom
0,2019-08-11,The Chicago Theatre,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2019-08-11,Main Hall in UIC Dorin Forum,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2019-08-11,Arie Crown Theater,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,2019-08-11,Lincoln Park Zoo,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,2019-08-11,Cinespace Chicago Film Studios,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9932,2018-07-27,Concord Music Hall,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
9933,2018-07-27,Soldier Field,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
9934,2018-07-27,Credit Union 1 Arena,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
9935,2018-07-27,Huntington Bank Pavilion,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [107]:
relevant_crimes["day"] = relevant_crimes.date.dt.date
crimes_by_date = relevant_crimes.groupby(by=['day']).sum()[locations].reset_index()
crimes_by_date["day"] = pd.to_datetime(crimes_by_date["day"])
crimes_by_date.rename(columns={'day':'date'}, inplace=True)
crimes_by_date

/project/graziul/ra/drewkeller/geoconda/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,date,The Chicago Theatre,Main Hall in UIC Dorin Forum,Arie Crown Theater,Lincoln Park Zoo,Cinespace Chicago Film Studios,Millennium Park,Petrillo Music Shell,Grant Park,Field Museum,...,Riviera Theatre,Gallagher Way,Guaranteed Rate Field,Sheraton Grand Chicago,Wrigley Field,Concord Music Hall,Soldier Field,Credit Union 1 Arena,Huntington Bank Pavilion,Aragon Ballroom
0,2018-07-27,43,2,10,4,6,33,20,13,8,...,8,10,4,32,14,6,10,5,1,8
1,2018-07-28,34,4,3,6,8,27,23,16,8,...,11,12,11,34,12,9,13,9,2,11
2,2018-07-29,45,6,8,6,6,32,22,5,3,...,3,21,5,33,21,11,6,6,0,3
3,2018-07-30,43,8,8,6,8,38,35,14,10,...,6,12,3,33,13,4,16,4,0,6
4,2018-07-31,41,4,4,5,5,33,19,12,9,...,12,6,4,33,8,7,12,10,0,11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
376,2019-08-07,34,4,4,7,7,30,23,11,8,...,10,9,7,30,7,5,11,6,1,9
377,2019-08-08,42,7,6,2,5,38,25,14,8,...,16,8,6,29,11,7,13,5,0,17
378,2019-08-09,34,9,6,6,6,31,21,9,4,...,8,11,4,33,9,7,6,13,0,7
379,2019-08-10,41,5,7,3,8,29,15,7,5,...,10,17,6,33,16,6,9,11,0,10


In [108]:
# check
data[(data["Arie Crown Theater"]==1) & (data.date.dt.date == dt.date.fromisoformat('2018-07-29'))]  # should be 8 per above

,Unnamed: 0,id,case_number,date,block,iucr,primary_type,description,location_description,arrest,...,8to11_crime,9to12_crime,event_evening,event_3to6,event_4to7,event_5to8,event_6to9,event_7to10,event_8to11,event_9to12
685,685,11395661,JB369820,2018-07-29 02:30:00,017XX S STATE ST,1210,DECEPTIVE PRACTICE,THEFT OF LABOR/SERVICES,TAXICAB,True,...,0,0,0,0,0,0,0,0,0,0
758,758,11400175,JB370130,2018-07-29 11:20:00,005XX W 18TH ST,0580,STALKING,SIMPLE,RESIDENCE,False,...,0,0,0,0,0,0,0,0,0,0
812,812,11396812,JB370327,2018-07-29 14:26:00,028XX S STATE ST,0486,BATTERY,DOMESTIC BATTERY SIMPLE,APARTMENT,False,...,0,0,0,0,0,0,0,0,0,0
875,875,11396606,JB370949,2018-07-29 18:15:00,025XX S MICHIGAN AVE,0460,BATTERY,SIMPLE,HOSPITAL BUILDING/GROUNDS,False,...,0,0,0,0,0,0,0,0,0,0
880,880,11396515,JB370925,2018-07-29 18:30:00,020XX S WABASH AVE,0810,THEFT,OVER $500,STREET,False,...,0,0,0,0,0,0,0,0,0,0
902,902,11396693,JB371175,2018-07-29 19:45:00,026XX S NORMAL AVE,0910,MOTOR VEHICLE THEFT,AUTOMOBILE,STREET,False,...,0,0,0,0,0,0,0,0,0,0
934,934,11396490,JB370844,2018-07-29 21:36:00,021XX S TAN CT,0330,ROBBERY,AGGRAVATED,SIDEWALK,True,...,1,1,0,0,0,0,0,0,0,0
948,948,11396519,JB370843,2018-07-29 22:07:00,023XX S PRINCETON AVE,0340,ROBBERY,ATTEMPT: STRONGARM-NO WEAPON,RESIDENTIAL YARD (FRONT/BACK),True,...,1,1,0,0,0,0,0,0,0,0


In [109]:
# assign proper crimes from crimes_by_date to relevant_crimes_daily
relevant_crimes_daily = pd.merge(relevant_crimes_daily, crimes_by_date, on=['date'], how='left')
relevant_crimes_daily

,date,Location,event,The Chicago Theatre_x,Main Hall in UIC Dorin Forum_x,Arie Crown Theater_x,Lincoln Park Zoo_x,Cinespace Chicago Film Studios_x,Millennium Park_x,Petrillo Music Shell_x,...,Riviera Theatre_y,Gallagher Way_y,Guaranteed Rate Field_y,Sheraton Grand Chicago_y,Wrigley Field_y,Concord Music Hall_y,Soldier Field_y,Credit Union 1 Arena_y,Huntington Bank Pavilion_y,Aragon Ballroom_y
0,2019-08-11,The Chicago Theatre,0,1,0,0,0,0,0,0,...,0,2,0,0,3,0,0,0,0,0
1,2019-08-11,Main Hall in UIC Dorin Forum,0,0,1,0,0,0,0,0,...,0,2,0,0,3,0,0,0,0,0
2,2019-08-11,Arie Crown Theater,0,0,0,1,0,0,0,0,...,0,2,0,0,3,0,0,0,0,0
3,2019-08-11,Lincoln Park Zoo,0,0,0,0,1,0,0,0,...,0,2,0,0,3,0,0,0,0,0
4,2019-08-11,Cinespace Chicago Film Studios,0,0,0,0,0,1,0,0,...,0,2,0,0,3,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9901,2018-07-27,Concord Music Hall,0,0,0,0,0,0,0,0,...,8,10,4,32,14,6,10,5,1,8
9902,2018-07-27,Soldier Field,0,0,0,0,0,0,0,0,...,8,10,4,32,14,6,10,5,1,8
9903,2018-07-27,Credit Union 1 Arena,0,0,0,0,0,0,0,0,...,8,10,4,32,14,6,10,5,1,8
9904,2018-07-27,Huntington Bank Pavilion,0,0,0,0,0,0,0,0,...,8,10,4,32,14,6,10,5,1,8


In [110]:
relevant_crimes_daily["day_crimes"] = 0
for location in locations:
    relevant_crimes_daily["day_crimes"] = np.where(
        relevant_crimes_daily.Location==location,
        relevant_crimes_daily[location+'_y'],
        relevant_crimes_daily["day_crimes"])
relevant_crimes_daily

,date,Location,event,The Chicago Theatre_x,Main Hall in UIC Dorin Forum_x,Arie Crown Theater_x,Lincoln Park Zoo_x,Cinespace Chicago Film Studios_x,Millennium Park_x,Petrillo Music Shell_x,...,Gallagher Way_y,Guaranteed Rate Field_y,Sheraton Grand Chicago_y,Wrigley Field_y,Concord Music Hall_y,Soldier Field_y,Credit Union 1 Arena_y,Huntington Bank Pavilion_y,Aragon Ballroom_y,day_crimes
0,2019-08-11,The Chicago Theatre,0,1,0,0,0,0,0,0,...,2,0,0,3,0,0,0,0,0,0
1,2019-08-11,Main Hall in UIC Dorin Forum,0,0,1,0,0,0,0,0,...,2,0,0,3,0,0,0,0,0,0
2,2019-08-11,Arie Crown Theater,0,0,0,1,0,0,0,0,...,2,0,0,3,0,0,0,0,0,0
3,2019-08-11,Lincoln Park Zoo,0,0,0,0,1,0,0,0,...,2,0,0,3,0,0,0,0,0,0
4,2019-08-11,Cinespace Chicago Film Studios,0,0,0,0,0,1,0,0,...,2,0,0,3,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9901,2018-07-27,Concord Music Hall,0,0,0,0,0,0,0,0,...,10,4,32,14,6,10,5,1,8,6
9902,2018-07-27,Soldier Field,0,0,0,0,0,0,0,0,...,10,4,32,14,6,10,5,1,8,10
9903,2018-07-27,Credit Union 1 Arena,0,0,0,0,0,0,0,0,...,10,4,32,14,6,10,5,1,8,5
9904,2018-07-27,Huntington Bank Pavilion,0,0,0,0,0,0,0,0,...,10,4,32,14,6,10,5,1,8,1


In [111]:
relevant_crimes_daily = relevant_crimes_daily[['date', 'Location', 'event','day_crimes', 'Thalia Hall_x',
       'Huntington Bank Pavilion_x', 'Union Station_x',
       'Cinespace Chicago Film Studios_x', 'Lincoln Park Zoo_x',
       'Chicago Symphony Orchestra_x', 'Petrillo Music Shell_x',
       'Millennium Park_x', 'Gallagher Way_x', 'Wrigley Field_x',
       'Concord Music Hall_x', 'Sheraton Grand Chicago_x',
       'The Chicago Theatre_x', 'Aragon Ballroom_x', 'Auditorium Theatre_x',
       'Museum of Science and Industry_x', 'Field Museum_x', 'Grant Park_x',
       'Main Hall in UIC Dorin Forum_x', 'Arie Crown Theater_x',
       'Soldier Field_x', 'Civic Opera House_x', 'United Center_x',
       'Guaranteed Rate Field_x', 'Credit Union 1 Arena_x',
       'Riviera Theatre_x']]

In [112]:
for column in relevant_crimes_daily.columns:
    if column.endswith('_x'):
        relevant_crimes_daily = relevant_crimes_daily.rename(columns={column:column[:-2]})
relevant_crimes_daily

,date,Location,event,day_crimes,Thalia Hall,Huntington Bank Pavilion,Union Station,Cinespace Chicago Film Studios,Lincoln Park Zoo,Chicago Symphony Orchestra,...,Field Museum,Grant Park,Main Hall in UIC Dorin Forum,Arie Crown Theater,Soldier Field,Civic Opera House,United Center,Guaranteed Rate Field,Credit Union 1 Arena,Riviera Theatre
0,2019-08-11,The Chicago Theatre,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2019-08-11,Main Hall in UIC Dorin Forum,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
2,2019-08-11,Arie Crown Theater,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
3,2019-08-11,Lincoln Park Zoo,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
4,2019-08-11,Cinespace Chicago Film Studios,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9901,2018-07-27,Concord Music Hall,0,6,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9902,2018-07-27,Soldier Field,0,10,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
9903,2018-07-27,Credit Union 1 Arena,0,5,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
9904,2018-07-27,Huntington Bank Pavilion,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [114]:
# now make this a function and add other crime types

def add_crimes(relevant_crimes_daily,relevant_crimes,crime_type):
    crimes_by_date = relevant_crimes[relevant_crimes[crime_type]>0].groupby(by=['day']).sum()[locations].reset_index()
    crimes_by_date["day"] = pd.to_datetime(crimes_by_date["day"])
    crimes_by_date.rename(columns={'day':'date'}, inplace=True)
    relevant_crimes_daily = pd.merge(relevant_crimes_daily, crimes_by_date, on=['date'], how='left')
    relevant_crimes_daily = relevant_crimes_daily.fillna(0)
    relevant_crimes_daily[crime_type] = 0
    for location in locations:
        relevant_crimes_daily[crime_type] = np.where(
            relevant_crimes_daily.Location==location,
            relevant_crimes_daily[location+'_y'],
            relevant_crimes_daily[crime_type])
    relevant_crimes_daily = relevant_crimes_daily.drop(columns=['Petrillo Music Shell_y', 'Grant Park_y', 'Guaranteed Rate Field_y',
       'United Center_y', 'Lincoln Park Zoo_y', 'Riviera Theatre_y',
       'Thalia Hall_y', 'Millennium Park_y', 'Field Museum_y',
       'Concord Music Hall_y', 'Wrigley Field_y', 'Auditorium Theatre_y',
       'Huntington Bank Pavilion_y', 'Union Station_y',
       'Main Hall in UIC Dorin Forum_y', 'Museum of Science and Industry_y',
       'Credit Union 1 Arena_y', 'Civic Opera House_y', 'Soldier Field_y',
       'Sheraton Grand Chicago_y', 'Cinespace Chicago Film Studios_y',
       'Gallagher Way_y', 'Arie Crown Theater_y', 'The Chicago Theatre_y',
       'Chicago Symphony Orchestra_y', 'Aragon Ballroom_y',])
    for column in relevant_crimes_daily.columns:
        if column.endswith('_x'):
            relevant_crimes_daily = relevant_crimes_daily.rename(columns={column:column[:-2]})
    
    return relevant_crimes_daily

event_types = [
    "day_crimes","evening_crime","3to6_crime","4to7_crime","5to8_crime",
    "6to9_crime","7to10_crime","8to11_crime","9to12_crime"
    ]
for crime_type in event_types[1:]:
    relevant_crimes_daily = add_crimes(relevant_crimes_daily,relevant_crimes,crime_type)
relevant_crimes_daily

,date,Location,event,day_crimes,Thalia Hall,Huntington Bank Pavilion,Union Station,Cinespace Chicago Film Studios,Lincoln Park Zoo,Chicago Symphony Orchestra,...,Credit Union 1 Arena,Riviera Theatre,evening_crime,3to6_crime,4to7_crime,5to8_crime,6to9_crime,7to10_crime,8to11_crime,9to12_crime
0,2019-08-11,The Chicago Theatre,0,0,0,0,0,0,0,0,...,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2019-08-11,Main Hall in UIC Dorin Forum,0,0,0,0,0,0,0,0,...,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2019-08-11,Arie Crown Theater,0,0,0,0,0,0,0,0,...,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2019-08-11,Lincoln Park Zoo,0,0,0,0,0,0,1,0,...,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2019-08-11,Cinespace Chicago Film Studios,0,0,0,0,0,1,0,0,...,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9901,2018-07-27,Concord Music Hall,0,6,0,0,0,0,0,0,...,0,0,1.0,2.0,2.0,1.0,0.0,0.0,1.0,1.0
9902,2018-07-27,Soldier Field,0,10,0,0,0,0,0,0,...,0,0,3.0,0.0,2.0,2.0,3.0,3.0,1.0,1.0
9903,2018-07-27,Credit Union 1 Arena,0,5,0,0,0,0,0,0,...,1,0,2.0,0.0,0.0,0.0,0.0,2.0,2.0,2.0
9904,2018-07-27,Huntington Bank Pavilion,0,1,0,1,0,0,0,0,...,0,0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0


In [127]:
# check
relevant_crimes_daily[(
    relevant_crimes_daily.Location=="Arie Crown Theater") & (
    relevant_crimes_daily.date.dt.date == dt.date.fromisoformat('2018-07-29'))]  # should be 8 per above
# also, from inspecting above we can see evening_crime should be 5, for example

,date,Location,event,day_crimes,Thalia Hall,Huntington Bank Pavilion,Union Station,Cinespace Chicago Film Studios,Lincoln Park Zoo,Chicago Symphony Orchestra,...,Credit Union 1 Arena,Riviera Theatre,evening_crime,3to6_crime,4to7_crime,5to8_crime,6to9_crime,7to10_crime,8to11_crime,9to12_crime
9830,2018-07-29,Arie Crown Theater,0,8,0,0,0,0,0,0,...,0,0,5.0,2.0,3.0,3.0,4.0,3.0,2.0,2.0


Now we have a dataframe called `relevant_crimes_daily` which contains one observation per venue per day. The indicator `event` is 1 if there was at least one event at that venue that day, else 0. The `day_crimes`, `evening_crime`, `3to6_crime`, etc. variables list the number of crimes that occurred within the relevant spatial and temporal bandwidth. (Note that the sum of crimes in this dataframe is more than the number of crimes in `relevant_crimes` (all within-bandwidth crimes), because crimes are logged for each venue they're within the bandwidth of. This also means that we might get different-direction results from a simple regression than we did from the difference in means, as we will see below.)

Now we'll first try a simple linear regression of daily crimes on whether there was an event nearby. This could recover an unbiased estimate of the effect of events on crimes if venues had been completely randomly assigned to have events - which they obviously did not.

In [128]:
X = relevant_crimes_daily.event
Y = relevant_crimes_daily.day_crimes

# sklearn version
#day_regression = linear_model.LinearRegression()
#day_regression.fit(X,Y)

print('Same day - simple regression:')
sm.OLS(Y, X).fit().summary()

Same day - simple regression:


<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:             day_crimes   R-squared (uncentered):                   0.042
Model:                            OLS   Adj. R-squared (uncentered):              0.041
Method:                 Least Squares   F-statistic:                              429.9
Date:                Mon, 20 Jun 2022   Prob (F-statistic):                    1.64e-93
Time:                        18:46:08   Log-Likelihood:                         -42573.
No. Observations:                9906   AIC:                                  8.515e+04
Df Residuals:                    9905   BIC:                                  8.515e+04
Df Model:                           1                                                  
Covariance Type:            nonrobust                                                  
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
event         12.3449      0.595     20.733      0.000      11.178      13.512
==============================================================================
Omnibus:                     3404.642   Durbin-Watson:                   1.165
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            15920.253
Skew:                           1.611   Prob(JB):                         0.00
Kurtosis:                       8.309   Cond. No.                         1.00
==============================================================================

Notes:
[1] R² is computed without centering (uncentered) since the model does not contain a constant.
[2] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

This estimates that having an event causes, on average, 12 more crimes that day within a 2km square around the venue. However, this is certainly biased due to our lack of actually being able to randomize events; any factors that both make crimes and events more likely would induce upward bias on our estimate (for example, day of the week, or venue type).

To try to somewhat account for this we will now repeat the regression with venue fixed effects and then venue and date fixed effects.

In [129]:
X = relevant_crimes_daily.drop(columns=['Location','date']+event_types)
Y = relevant_crimes_daily.day_crimes

print('Same-day crime - venue fixed effects:')
sm.OLS(Y, X).fit().summary()

Same-day crime - venue fixed effects:


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:             day_crimes   R-squared:                       0.759
Model:                            OLS   Adj. R-squared:                  0.759
Method:                 Least Squares   F-statistic:                     1200.
Date:                Mon, 20 Jun 2022   Prob (F-statistic):               0.00
Time:                        18:46:13   Log-Likelihood:                -32022.
No. Observations:                9906   AIC:                         6.410e+04
Df Residuals:                    9879   BIC:                         6.429e+04
Df Model:                          26                                         
Covariance Type:            nonrobust                                         
==================================================================================================
                                     coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------------
event                              0.1228      0.227      0.540      0.589      -0.323       0.568
Thalia Hall                        4.8968      0.319     15.370      0.000       4.272       5.521
Huntington Bank Pavilion           0.3607      0.315      1.146      0.252      -0.257       0.978
Union Station                     16.1913      0.315     51.463      0.000      15.575      16.808
Cinespace Chicago Film Studios     6.3330      0.315     20.129      0.000       5.716       6.950
Lincoln Park Zoo                   3.8767      0.315     12.316      0.000       3.260       4.494
Chicago Symphony Orchestra        27.7237      0.316     87.708      0.000      27.104      28.343
Petrillo Music Shell              23.3553      0.315     74.230      0.000      22.739      23.972
Millennium Park                   31.8953      0.316    100.810      0.000      31.275      32.516
Gallagher Way                     10.0755      0.317     31.825      0.000       9.455      10.696
Wrigley Field                     10.4201      0.315     33.085      0.000       9.803      11.037
Concord Music Hall                 6.5276      0.323     20.213      0.000       5.895       7.161
Sheraton Grand Chicago            32.7894      0.315    104.218      0.000      32.173      33.406
The Chicago Theatre               38.5281      0.315    122.247      0.000      37.910      39.146
Aragon Ballroom                    8.7254      0.317     27.543      0.000       8.104       9.346
Auditorium Theatre                24.2813      0.315     77.103      0.000      23.664      24.899
Museum of Science and Industry     3.1720      0.315     10.079      0.000       2.555       3.789
Field Museum                       5.2510      0.315     16.690      0.000       4.634       5.868
Grant Park                        10.6982      0.315     33.999      0.000      10.081      11.315
Main Hall in UIC Dorin Forum       5.6191      0.315     17.860      0.000       5.002       6.236
Arie Crown Theater                 5.5069      0.315     17.502      0.000       4.890       6.124
Soldier Field                      8.4549      0.315     26.830      0.000       7.837       9.073
Civic Opera House                 31.2872      0.315     99.294      0.000      30.670      31.905
United Center                      6.9210      0.324     21.349      0.000       6.286       7.557
Guaranteed Rate Field              4.3495      0.315     13.819      0.000       3.733       4.966
Credit Union 1 Arena               6.5718      0.316     20.822      0.000       5.953       7.190
Riviera Theatre                    8.7953      0.316     27.793      0.000       8.175       9.416
==============================================================================
Omnibus:                     9107.

In [130]:
time_dummies = relevant_crimes_daily.copy(deep=True).drop(columns='Location')
time_dummies["date"]=time_dummies.date.astype(str)
time_dummies = pd.get_dummies(time_dummies)
time_dummies

,event,day_crimes,Thalia Hall,Huntington Bank Pavilion,Union Station,Cinespace Chicago Film Studios,Lincoln Park Zoo,Chicago Symphony Orchestra,Petrillo Music Shell,Millennium Park,...,date_2019-08-02,date_2019-08-03,date_2019-08-04,date_2019-08-05,date_2019-08-06,date_2019-08-07,date_2019-08-08,date_2019-08-09,date_2019-08-10,date_2019-08-11
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9901,0,6,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9902,0,10,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9903,0,5,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9904,0,1,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [131]:
X = time_dummies.drop(columns=event_types+locations)
Y = time_dummies.day_crimes

print('Same-day crime - date fixed effects:')
sm.OLS(Y, X).fit().summary()

Same-day crime - date fixed effects:


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:             day_crimes   R-squared:                       0.066
Model:                            OLS   Adj. R-squared:                  0.029
Method:                 Least Squares   F-statistic:                     1.765
Date:                Mon, 20 Jun 2022   Prob (F-statistic):           1.61e-17
Time:                        18:46:21   Log-Likelihood:                -38741.
No. Observations:                9906   AIC:                         7.825e+04
Df Residuals:                    9524   BIC:                         8.100e+04
Df Model:                         381                                         
Covariance Type:            nonrobust                                         
===================================================================================
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
event              -1.1094      0.446     -2.489      0.013      -1.983      -0.236
date_2018-07-27    14.2308      2.417      5.888      0.000       9.493      18.969
date_2018-07-28    14.0042      2.417      5.794      0.000       9.266      18.742
date_2018-07-29    13.8077      2.417      5.713      0.000       9.070      18.546
date_2018-07-30    15.7692      2.417      6.524      0.000      11.031      20.507
date_2018-07-31    14.6538      2.417      6.063      0.000       9.916      19.392
date_2018-08-01    17.3203      2.418      7.164      0.000      12.581      22.059
date_2018-08-02    23.8546      2.417      9.868      0.000      19.116      28.593
date_2018-08-03    27.9399      2.418     11.555      0.000      23.200      32.680
date_2018-08-04    26.7818      2.418     11.078      0.000      22.043      31.521
date_2018-08-05    22.6581      2.417      9.374      0.000      17.920      27.396
date_2018-08-06    14.2734      2.417      5.905      0.000       9.535      19.012
date_2018-08-07    13.7392      2.417      5.684      0.000       9.001      18.478
date_2018-08-08    14.0811      2.417      5.826      0.000       9.343      18.819
date_2018-08-09    11.3546      2.417      4.697      0.000       6.616      16.093
date_2018-08-10    17.5084      2.417      7.243      0.000      12.770      22.247
date_2018-08-11    13.8203      2.418      5.716      0.000       9.081      18.559
date_2018-08-12    12.7776      2.417      5.286      0.000       8.039      17.516
date_2018-08-13    14.0469      2.417      5.811      0.000       9.308      18.785
date_2018-08-14    15.1749      2.419      6.274      0.000      10.434      19.916
date_2018-08-15    14.1581      2.417      5.857      0.000       9.420      18.896
date_2018-08-16    16.3546      2.417      6.766      0.000      11.616      21.093
date_2018-08-17    12.6581      2.417      5.237      0.000       7.920      17.396
date_2018-08-18    15.7861      2.418      6.528      0.000      11.046      20.526
date_2018-08-19    12.2007      2.417      5.047      0.000       7.462      16.939
date_2018-08-20    12.0811      2.417      4.998      0.000       7.343      16.819
date_2018-08-21    14.6238      2.417      6.050      0.000       9.885      19.362
date_2018-08-22    14.8119      2.417      6.128      0.000      10.074      19.550
date_2018-08-23    14.7692      2.417      6.110      0.000      10.031      19.507
date_2018-08-24    13.9784      2.418      5.781      0.000       9.238      18.718
date_2018-08-25    12.4399      2.418      5.145      0.000       7.700      17.180
date_2018-08-26    11.5769      2.417      4.790      0.000       6.839      16.315
date_2018-08-27    15.1196      2.417      6.255      0.000      10.381      19.858
date_2018-08-28    12.9700      2.417      5.365      0.000

We shouldn't pay too much attention to these, though - considering all the crimes from the day is certainly much too high a time bandwidth that could hide any real effect.

We'll try the same thing with crimes from 6pm - midnight on the day of an event.

In [133]:
X = relevant_crimes_daily.event
Y = relevant_crimes_daily.evening_crime

print('Evening - simple regression:')
sm.OLS(Y, X).fit().summary()

Evening - simple regression:


<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:          evening_crime   R-squared (uncentered):                   0.033
Model:                            OLS   Adj. R-squared (uncentered):              0.033
Method:                 Least Squares   F-statistic:                              338.0
Date:                Mon, 20 Jun 2022   Prob (F-statistic):                    3.01e-74
Time:                        18:46:33   Log-Likelihood:                         -31197.
No. Observations:                9906   AIC:                                  6.240e+04
Df Residuals:                    9905   BIC:                                  6.240e+04
Df Model:                           1                                                  
Covariance Type:            nonrobust                                                  
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
event          3.4714      0.189     18.384      0.000       3.101       3.842
==============================================================================
Omnibus:                    11477.017   Durbin-Watson:                   1.246
Prob(Omnibus):                  0.000   Jarque-Bera (JB):          2321745.276
Skew:                           5.834   Prob(JB):                         0.00
Kurtosis:                      77.087   Cond. No.                         1.00
==============================================================================

Notes:
[1] R² is computed without centering (uncentered) since the model does not contain a constant.
[2] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [135]:
X = relevant_crimes_daily.drop(columns=['Location','date']+event_types)
Y = relevant_crimes_daily.evening_crime

print('Evening - venue fixed effects:')
sm.OLS(Y, X).fit().summary()

Evening - venue fixed effects:


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          evening_crime   R-squared:                       0.371
Model:                            OLS   Adj. R-squared:                  0.370
Method:                 Least Squares   F-statistic:                     224.5
Date:                Mon, 20 Jun 2022   Prob (F-statistic):               0.00
Time:                        18:46:44   Log-Likelihood:                -26570.
No. Observations:                9906   AIC:                         5.319e+04
Df Residuals:                    9879   BIC:                         5.339e+04
Df Model:                          26                                         
Covariance Type:            nonrobust                                         
==================================================================================================
                                     coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------------
event                             -0.0010      0.131     -0.008      0.994      -0.258       0.256
Thalia Hall                        1.5567      0.184      8.472      0.000       1.196       1.917
Huntington Bank Pavilion           0.1444      0.182      0.795      0.427      -0.212       0.500
Union Station                      3.6929      0.181     20.351      0.000       3.337       4.049
Cinespace Chicago Film Studios     1.9764      0.181     10.891      0.000       1.621       2.332
Lincoln Park Zoo                   1.1890      0.182      6.549      0.000       0.833       1.545
Chicago Symphony Orchestra         7.0999      0.182     38.944      0.000       6.743       7.457
Petrillo Music Shell               6.2651      0.181     34.524      0.000       5.909       6.621
Millennium Park                    8.4175      0.182     46.128      0.000       8.060       8.775
Gallagher Way                      3.0002      0.183     16.431      0.000       2.642       3.358
Wrigley Field                      3.1418      0.182     17.296      0.000       2.786       3.498
Concord Music Hall                 2.1946      0.186     11.782      0.000       1.829       2.560
Sheraton Grand Chicago             8.7953      0.181     48.469      0.000       8.440       9.151
The Chicago Theatre                9.8584      0.182     54.234      0.000       9.502      10.215
Aragon Ballroom                    2.5697      0.183     14.064      0.000       2.212       2.928
Auditorium Theatre                 6.2888      0.182     34.623      0.000       5.933       6.645
Museum of Science and Industry     0.8845      0.182      4.873      0.000       0.529       1.240
Field Museum                       1.6667      0.181      9.185      0.000       1.311       2.022
Grant Park                         3.0735      0.181     16.936      0.000       2.718       3.429
Main Hall in UIC Dorin Forum       1.6325      0.181      8.997      0.000       1.277       1.988
Arie Crown Theater                 1.5538      0.181      8.562      0.000       1.198       1.910
Soldier Field                      2.5565      0.182     14.066      0.000       2.200       2.913
Civic Opera House                  7.8190      0.182     43.024      0.000       7.463       8.175
United Center                      2.4492      0.187     13.099      0.000       2.083       2.816
Guaranteed Rate Field              1.3701      0.182      7.547      0.000       1.014       1.726
Credit Union 1 Arena               2.0474      0.182     11.247      0.000       1.691       2.404
Riviera Theatre                    2.5750      0.183     14.108      0.000       2.217       2.933
==============================================================================
Omnibus:                    15084.

In [136]:
X = time_dummies.drop(columns=event_types+locations)
Y = time_dummies.evening_crime

print('Evening crimes - date fixed effects:')
sm.OLS(Y, X).fit().summary()

Evening crimes - date fixed effects:


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          evening_crime   R-squared:                       0.155
Model:                            OLS   Adj. R-squared:                  0.122
Method:                 Least Squares   F-statistic:                     4.600
Date:                Mon, 20 Jun 2022   Prob (F-statistic):          1.94e-155
Time:                        18:46:58   Log-Likelihood:                -28034.
No. Observations:                9906   AIC:                         5.683e+04
Df Residuals:                    9524   BIC:                         5.958e+04
Df Model:                         381                                         
Covariance Type:            nonrobust                                         
===================================================================================
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
event              -0.1649      0.151     -1.091      0.275      -0.461       0.131
date_2018-07-27     3.8077      0.820      4.643      0.000       2.200       5.415
date_2018-07-28     4.0833      0.820      4.979      0.000       2.476       5.691
date_2018-07-29     3.2308      0.820      3.940      0.000       1.623       4.838
date_2018-07-30     3.2308      0.820      3.940      0.000       1.623       4.838
date_2018-07-31     4.4615      0.820      5.440      0.000       2.854       6.069
date_2018-08-01     4.2113      0.820      5.134      0.000       2.603       5.819
date_2018-08-02     8.3204      0.820     10.145      0.000       6.713       9.928
date_2018-08-03    11.4100      0.820     13.908      0.000       9.802      13.018
date_2018-08-04    11.8652      0.820     14.465      0.000      10.257      13.473
date_2018-08-05     7.5448      0.820      9.200      0.000       5.937       9.152
date_2018-08-06     2.5833      0.820      3.150      0.002       0.976       4.191
date_2018-08-07     3.1281      0.820      3.814      0.000       1.520       4.736
date_2018-08-08     3.4679      0.820      4.229      0.000       1.860       5.075
date_2018-08-09     2.4742      0.820      3.017      0.003       0.867       4.082
date_2018-08-10     4.4358      0.820      5.408      0.000       2.828       6.043
date_2018-08-11     3.2113      0.820      3.915      0.000       1.603       4.819
date_2018-08-12     4.5896      0.820      5.596      0.000       2.982       6.197
date_2018-08-13     3.0511      0.820      3.720      0.000       1.443       4.659
date_2018-08-14     6.9163      0.821      8.429      0.000       5.308       8.525
date_2018-08-15     4.3525      0.820      5.307      0.000       2.745       5.960
date_2018-08-16     4.2819      0.820      5.221      0.000       2.674       5.890
date_2018-08-17     4.0448      0.820      4.932      0.000       2.437       5.652
date_2018-08-18     4.9100      0.820      5.985      0.000       3.302       6.518
date_2018-08-19     3.4358      0.820      4.189      0.000       1.828       5.043
date_2018-08-20     3.2756      0.820      3.994      0.000       1.668       4.883
date_2018-08-21     4.3588      0.820      5.315      0.000       2.751       5.967
date_2018-08-22     4.6987      0.820      5.729      0.000       3.091       6.306
date_2018-08-23     3.5000      0.820      4.268      0.000       1.892       5.108
date_2018-08-24     3.5638      0.820      4.344      0.000       1.956       5.172
date_2018-08-25     2.8715      0.820      3.500      0.000       1.263       4.480
date_2018-08-26     3.2692      0.820      3.987      0.000       1.662       4.877
date_2018-08-27     4.2371      0.820      5.167      0.000       2.630       5.845
date_2018-08-28     3.7435      0.820      4.564      0.000

Without venue or time fixed effects, we estimate that having an event on the same day increases evening crimes within a 2 km square by more than 3 per event (remember that we are treating multiple nearby events' effects as linearly additive), which is both statistically and practically significant (remember that on average there are only 3 to 5 crimes logged per evening within a 2 km square of all the venues). Also, the likely-larger-than-ideal time bandwidth would probably attenuate this estimate from a real effect. 

On the other hand, this estimates is very susceptible to bias from selection. Notably, adding either venue or time fixed effects results in an estimated effect that is very small and we cannot reject the null hypothesis.

Finally, we'll estimate time fixed effects regressions of crimes within each 3-hour time window on same-day events.

In [151]:
results = {}
for event_type in event_types[2:]:
    X = time_dummies.drop(columns=event_types+locations)
    Y = time_dummies[event_type]
    model = sm.OLS(Y, X).fit()
    results[event_type] = [model.params[0],model.pvalues[0]]
print('Estimated effect of event on crimes by time period (and p values)')
results

Estimated effect of event on crimes by time period (and p values)


{'3to6_crime': [-0.5210523668192552, 0.0003645583788985817],
 '4to7_crime': [-0.41445746767945946, 0.0031271402272106865],
 '5to8_crime': [-0.3364354343779856, 0.010756584941803805],
 '6to9_crime': [-0.20257558227274855, 0.09706204060034679],
 '7to10_crime': [-0.0934151617284556, 0.4085114470081045],
 '8to11_crime': [-0.02625886614014658, 0.7862465451523972],
 '9to12_crime': [0.0029176517933506563, 0.96648191606254]}

These results are a bit strange. The estimates start out significant (same-day events cause -0.52 crimes from 3 to 6pm, p = 0.0004) and reduce later in the evening.

Personally, I think these results suggest that we cannot detect a measurable effect of the kind of event in our dataset on crimes, but the high possibility of selection on unobservables means further analysis is warranted.